# 파이토치에서 일반적인 체크포인트(Checkpoint)를 저장하고 불러오기

추론(inference) 또는 학습(traning)의 재개를 위해 체크포인트(checkpoint) 모델을 저장하고 불러오는 것은 마지막으로 중단했던 부분을 선택하는데 도움을 준다.

체크포인트를 저장할 때는 단순히 모델의 state_dict 이상의 것을 저장해야 한다.

모델 학습중에 갱신되는 버퍼와 매개변수들을 포함하는 옵티마이저(optimizer)의 state_dict도 함께 저장해야 한다.

이 외에도 중단 시점의 에포크 마지막으로 기록된 오차, 외부 계층등의 정보도 함께 저장해야 한다.

## 1. 데이터 불러올 때 필요한 라이브러리 불러오기

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## 2. 신경망을 구성하고 초기화하기

In [3]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3,6,5)
    self.conv2 = nn.Conv2d(6,16,5)
    
    self.pool = nn.MaxPool2d(2,2)
    
    self.cf1 = nn.Linear(16 * 5 * 5, 120)
    self.cf2 = nn.Linear(120, 84)
    self.cf3 = nn.Linear(84, 10)
    
  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16 * 5 * 5)
    x = F.relu(self.cf1(x))
    x = F.relu(self.cf2(x))
    x = self.cf3(x)
    return x
  
net = Net()
print(net)
    

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cf1): Linear(in_features=400, out_features=120, bias=True)
  (cf2): Linear(in_features=120, out_features=84, bias=True)
  (cf3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [5]:
# 추가 정보
EPOCH = 5
PATH = "model.pt"
LOSS = 0.4

torch.save({
            'epoch': EPOCH,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': LOSS,
            }, PATH)

## 5. 일반적인 체크포인트 불러오기

In [6]:
import torch.optim as optim
model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()

model.train()

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cf1): Linear(in_features=400, out_features=120, bias=True)
  (cf2): Linear(in_features=120, out_features=84, bias=True)
  (cf3): Linear(in_features=84, out_features=10, bias=True)
)